In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

periods = 12
horizon = 6 #plan in p1 for 1-6, in p2 for 2-7, etc. => plan, update demand, plan for next horizon
runs = periods-horizon+1

demand = [66, 78, 57, 34, 5, 118, 76, 61, 23, 31, 68, 48]
forecast = 50
initial_inv = 0

setupcost = 450
holdingcost = 2
backordercost = 20
#safetystock = 50
Big = sum(demand[t] for t in range(periods))

quantity = np.zeros(periods)

for k in range(runs): #same as before, rolling over horizon
  m = gp.Model()
  lotsize = {}
  inventory ={}
  setup = {}
  for t in range(horizon):
     lotsize[t] = m.addVar()
     inventory[t] = m.addVar(obj=holdingcost)
     setup[t] = m.addVar(vtype=GRB.BINARY, obj=setupcost)
  m.addConstr(inventory[0] == initial_inv+lotsize[0]-forecast)
  for t in range(1,horizon):
     m.addConstr(inventory[t-1]+lotsize[t]-forecast == inventory[t])
  for t in range(horizon):
     m.addConstr(lotsize[t] <= Big*setup[t])
# add safety stock
# for t in range(horizon):
#    m.addConstr(inventory[t] >= safetystock)

# should use less safety stock than frozen horizon because of horizon planning

  m.optimize()
  v = m.getVars()
  quantity[k] = v[0].x
  initial_inv = initial_inv+v[0].x-demand[k] #update initial inventory (balance) => endogenuous
  m.dispose

for k in range(horizon-1):
  quantity[runs+k] = v[3*(k+1)].x #do not plan for all 12 horizon , stop from period 6 
    
print(quantity)
  
cost = 0
inv = 0
for t in range(periods):
  if quantity[t] > 0:
    cost = cost+setupcost
  inv = inv+quantity[t]-demand[t]
  if inv > 0:
     cost=cost+holdingcost*inv
  else: cost=cost-backordercost*inv  
print("Rolling Horizon Cost: "+str(cost)) 


Using license file C:\Users\hadao\gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 12 rows, 18 columns and 29 nonzeros
Model fingerprint: 0x57d7f682
Variable types: 12 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [2e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 5e+01]
Presolve removed 3 rows and 4 columns
Presolve time: 0.01s
Presolved: 9 rows, 14 columns, 22 nonzeros
Variable types: 9 continuous, 5 integer (5 binary)

Root relaxation: objective 9.191729e+02, 6 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  919.17293    0    3          -  919.17293      -     -    0s
H    0     0                    2100.0000000  919.17293  56.2%     -    0s
H    0     0                 

 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  474.46617    0    3          -  474.46617      -     -    0s
H    0     0                    1658.0000000  474.46617  71.4%     -    0s
H    0     0                    1600.0000000  474.46617  70.3%     -    0s
H    0     0                    1458.0000000  474.46617  67.5%     -    0s
H    0     0                    1308.0000000  822.96241  37.1%     -    0s
     0     0 1308.00000    0    1 1308.00000 1308.00000  0.00%     -    0s

Cutting planes:
  Gomory: 1
  Implied bound: 5
  Clique: 1
  Flow cover: 2

Explored 1 nodes (11 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 4: 1308 1458 1600 1658 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.308000000000e+03, best bound 1.308000000000e+03, gap 0.0000%
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 12 rows, 18 columns and 29 nonzeros
Model fingerpri